# EDA Guía práctica usando Pandas

## Un análisis de los rendimientos de estudiantes en base a diferentes pruebas.

Original: https://towardsdatascience.com/practical-eda-guide-with-pandas-3463ea637609

Dataset: https://www.kaggle.com/spscientist/students-performance-in-exams

Pandas es una librería ampliamente utilizada para análisis de datos y proporciona numerosos métodos y funciones para trabajar con datos tabulares (filas y columnas, 2 dimensiones)

La valiosa selección de funciones **"easy-to-use"** hace el `Análisis Exploratorio de Datos (EDA)` un proceso sencillo.

Usaremos el dataset https://www.kaggle.com/spscientist/students-performance-in-exams. Éste dataset contiene información personal sobre estudiantes y sus rendimientos en varias pruebas.

### 1.- Leyendo el dataset

El primer paso es cargar las librerías que vamos a utilizar en éste estudio.

In [ ]:
# Librerías a utilizar
import numpy as np
import pandas as pd

Cargamos el dataset a un **dataframe** de Pandas.

In [ ]:
df = pd.read_csv("EDA_37_StudentsPerformance.csv")

Veamos cuantas filas y columnas tiene el dataset

In [ ]:
df.shape

El contenido de las primeras filas del dataframe es:

In [ ]:
df.head()

Comprobamos que hay 5 características categóricas y marcadores de 3 pruebas diferentes. El objetivo es comprobar que características influyen en dichos marcadores.

A continuación, comencemos a comprobar la distribución de los marcadores. Usaremos la función `plot`de Pandas para crear un `kernel density plot (KDE)`.

Para la prueba *reading* tenemos el siguiente gráfico:

In [ ]:
df['reading score'].plot(kind='kde', figsize=(10,6), title='Distribution of Reading Score')

Vemos que el marcador *reading* tiene una distribución normal.

Comprobemos el marcador *math*.

In [ ]:
df['reading score'].plot(kind='kde', figsize=(10,6), title='Distribution of Math Score')

Igualmente, tiene una distribución normal.

Hagamos el mismo gráfico para el marcador *writing*

In [ ]:
df['reading score'].plot(kind='kde', figsize=(10,6), title='Distribution of Writing Score')

La columna `race/ethnicity` contiene 5 grupos. Veamos el valor medio de cada marcador para cada grupo. Usaremos la función `groupby` de Pandas en ésta ocasión. Además, haremos un conteo de valores para cada marcador.

In [ ]:
df[['race/ethnicity','math score','reading score','writing score']].groupby('race/ethnicity').agg({'math score':'mean', 'reading score':'mean', 'writing score':['mean','count']}).round(1)

Si deseamos aplicar diferentes funciones agregadas a diferentes columnas, podemos pasar un diccionario a la función `agg`. Dado que el recuento es el mismo para todas las pruebas, es suficiente aplicar la función de recuento a una sola columna.

Los resultados muestran que los puntajes promedio aumentan constantemente del grupo A al grupo E.

Podemos usar la misma lógica para obtener una descripción general de la relación entre una variable categórica y los puntajes de las pruebas.

Para ir un paso más allá, se pueden crear grupos anidados. Hagamos un ejemplo con las columnas `genero`, `curso de preparación para exámenes` y `puntuación de matemáticas`. Los alumnos serán agrupados por género y cursando el curso de preparación. Luego, se calculará el puntaje promedio de matemáticas para cada grupo.

In [ ]:
df[['gender','test preparation course','math score']].groupby(['gender','test preparation course']).agg(['mean','count']).round(1)

El curso de preparación para el examen tiene un efecto positivo en los puntajes de matemáticas tanto para mujeres como para hombres. En general, los hombres obtienen mejores resultados que las mujeres en las pruebas de matemáticas.

Es posible que hayas notado la función de redondeo al final del código. Simplemente redondea los números de punto flotante al número deseado de puntos decimales.

También podemos comprobar cómo se relacionan los grupos en dos variables categóricas. Considera las columnas *raza/etnia* y *nivel de educación de los padres*. La distribución del nivel de educación puede ser diferente para cada grupo étnico.

Primero crearemos un dataframe que contenga la cantidad de personas para cada combinación de grupo étnico y nivel educativo.

In [ ]:
education = df[['race/ethnicity','parental level of education','lunch']].groupby(['race/ethnicity','parental level of education']).count().reset_index()
education.rename(columns={'lunch':'count'}, inplace=True)

education.head()

La tercera columna puede ser cualquier columna porque solo se usa para contar las observaciones (es decir, filas) que pertenecen a una combinación particular de etnia-nivel educativo. Esa es la razón por la que cambiamos el nombre de la columna de lunch a count.

El nivel de educación del "grupo A" está cerca de distribuirse uniformemente, excepto en el caso de la "maestría". Hay muchas opciones para visualizar el marco de datos educativo que acabamos de crear. 

Haremos un diagrama de línea polar de la biblioteca de gráficos **plotly**.

In [ ]:
import plotly.express as px

fig = px.line_polar(education, r='count', theta='parental level of education',color='race/ethnicity', line_close=True, width=800, height=500)
fig.show()

Nos da una visión general de la distribución del nivel educativo de los diferentes grupos. Es importante señalar que esta gráfica polar no se puede utilizar para comparar directamente los niveles de educación de diferentes grupos porque el número de estudiantes en cada grupo no es el mismo.

In [ ]:
df['race/ethnicity'].value_counts()

El grupo C contiene la mayor cantidad de estudiantes. Por lo tanto, usamos este gráfico polar para verificar la distribución del nivel de educación dentro de cada grupo.
Por ejemplo, "associate's degreee" es el nivel educativo más dominante en el grupo C. Podemos confirmar los resultados utilizando la función `value_counts`.

In [ ]:
education[education['race/ethnicity'] == 'group C'].value_counts()

Otra función útil para comparar categorías es `pivot_table`. Crea una tabla cruzada que consta de combinaciones de categorías.

Hagamos un ejemplo usando las columnas *lunch* y *parental level of education* en la puntuación *writing*.

In [ ]:
pd.pivot_table(df, values='writing score',index='parental level of education', columns='lunch',aggfunc='mean')

Las variables categóricas (es decir, columnas) se pasan a los parámetros de índice y columna y la variable numérica se pasa al parámetro de valores. Dependiendo de la función agregada seleccionada (parámetro `aggfunc`), se calculan los valores.

La tabla dinámica muestra que es probable que tener un `lunch` estándar aumente la puntuación *writing*. De manera similar, a medida que aumenta el nivel de educación de los padres, los estudiantes tienden a obtener mejores puntajes.

## Conclusion

Hemos descubierto algunas ideas sobre lo que afecta el desempeño de los estudiantes en las pruebas. Por supuesto, hay muchas medidas diferentes que puede verificar. Sin embargo, las técnicas son bastante similares.

Dependiendo de lo que busque, las técnicas que utilice se inclinarán hacia una dirección particular. Sin embargo, una vez que se sienta cómodo con el trabajo de Pandas, puede realizar prácticamente cualquier tarea con datos tabulares.